In [1]:
%load_ext autoreload
%autoreload 2

import os
import fitz
import boto3
import pandas as pd
from tqdm import tqdm
import sys

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.append(repo_root)

from src.utils import is_digitally_born

base_dir = os.getcwd()

"""Notebook to retrieve and save all text for all files on the S3 bucket as .txt files"""

'Notebook to retrieve and save all text for all files on the S3 bucket as .txt files'

In [2]:
s3_input = "asset/asset_files_new_ocr/"
s3_aws_profile = "779726271945_swissgeol-assets-ro"
s3_session = boto3.Session(profile_name=s3_aws_profile)
s3 = s3_session.resource('s3')
bucket = s3.Bucket('swissgeol-assets-swisstopo')

In [3]:
objs = list(bucket.objects.filter(Prefix=s3_input))

In [ ]:
output_dir = "./data/text_files/"

def key_to_filename(key):
    return key.split("/")[-1]

In [5]:
# count page number of pdfs and save to data
output_path = "./data/page_numbers.csv"
os.makedirs("./data", exist_ok=True)
page_data = []
with tqdm(total=len(objs)) as pbar:

    for obj in objs:
        pbar.set_postfix_str(obj.key)
        
        filename = key_to_filename(obj.key)
        if filename.endswith(".pdf"):
                    tmp_file_path = os.path.join("./data", filename)
                    bucket.download_file(obj.key, tmp_file_path)

                    doc = fitz.open(tmp_file_path)
                    
                    num_pages = len(doc)
                    #count digitally born pages
                    digital_pages = sum(is_digitally_born(page) for page in doc)

                    digital_ratio = digital_pages / num_pages
                    if digital_ratio >= 0.8:
                        category = "digital"
                    elif digital_ratio <= 0.2:
                        category = "scanned"
                    else:
                        category = "both"

                    page_data.append({
                        "filename": filename,
                        "pages": num_pages,
                        "digital_pages": digital_pages,
                        "scanned_pages": num_pages - digital_pages,
                        "category": category
                    })

                    os.remove(tmp_file_path)   

        
        pbar.update(1)
        
df = pd.DataFrame(page_data)
df.to_csv(output_path, index=False)

 75%|███████▍  | 11964/15997 [2:29:21<17:45,  3.79it/s, asset/asset_files_new_ocr/44229.pdf]       

MuPDF error: format error: No default Layer config



100%|██████████| 15997/15997 [2:51:48<00:00,  1.55it/s, asset/asset_files_new_ocr/a990_989_LDoc.pdf]                                                                                                  


In [ ]:
#write text of pdf into text files
with tqdm(total=len(objs)) as pbar:

    for obj in objs:
        pbar.set_postfix_str(obj.key)
        
        filename = key_to_filename(obj.key)
        if filename.endswith(".pdf"):
                    tmp_file_path = os.path.join("./data", filename)
                    bucket.download_file(obj.key, tmp_file_path)

                    doc = fitz.open(tmp_file_path)
                    output_path = os.path.join(output_dir, filename + ".txt")
                    
                    with open(output_path, "w", encoding = "utf-8") as text_file:
                        for _,page in enumerate(doc):
                            text= page.get_text()
                            text_file.write(text)

                    os.remove(tmp_file_path)   

        
        pbar.update(1)
